In [2]:
import torch
import torch.nn as nn
import numpy as np

In [3]:
import seaborn as sns
iris = sns.load_dataset('iris')


data = torch.tensor( iris[iris.columns[0:4]].values ).float()

labels = torch.zeros(len(data), dtype=torch.long)

labels[iris.species == 'setosa'] = 0
labels[iris.species == 'versicolor'] = 1
labels[iris.species == 'virginica'] = 2

In [19]:
labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [38]:
propTraining = 0.8
nTraining = int(len(labels)*propTraining)

traintestBool = np.zeros(len(labels), dtype=bool)

item2use4train = np.random.choice(range(len(labels)), nTraining, replace=False)
traintestBool[item2use4train] = True

traintestBool

array([False,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,
       False,  True,  True,  True,  True,  True, False,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True, False, False,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [39]:
# check whether it's balanced

print('Average of all data')
print(torch.mean(labels.float()).numpy(), end='\n\n')

print('Average of training data')
print(torch.mean(labels[traintestBool].float()).numpy(), end='\n\n')

print('Average of test data')
print(torch.mean(labels[~traintestBool].float()).numpy(), end='\n\n')

Average of all data
1.0

Average of training data
0.95

Average of test data
1.2



In [54]:
ANN = nn.Sequential(
    nn.Linear(4, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 3)
)

lossfun = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(ANN.parameters(), lr=0.01)

In [43]:
print(data.shape)
print(data[traintestBool].shape)
print(data[~traintestBool].shape)

torch.Size([150, 4])
torch.Size([120, 4])
torch.Size([30, 4])


In [55]:
numepochs = 1000

losses = torch.zeros(numepochs)
ongoingAcc = []

for epochi in range(numepochs):

    # forward pass
    yHat = ANN(data[traintestBool])

    # compute loss
    loss = lossfun(yHat, labels[traintestBool])
    losses[epochi] = loss

    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # compute accuracy
    matches = torch.argmax(yHat, axis=1) == labels[traintestBool] # booleans (true/false)
    matchesNumaric = matches.float()               # convet to numaric
    accuracyPct = 100*torch.mean(matchesNumaric)   # calculate acc 
    ongoingAcc.append(accuracyPct)

In [57]:
predictions = ANN(data[traintestBool])
trainacc = 100*(torch.mean((torch.argmax(predictions, axis=1) == labels[traintestBool]).float())).item()

predictions = ANN(data[~traintestBool])
testacc = 100*(torch.mean((torch.argmax(predictions, axis=1) == labels[~traintestBool]).float())).item()

In [58]:
trainacc, testacc

(98.33333492279053, 100.0)